# Spelling Correction Algorithms

The aim of this notebook is to illustrate spelling correction algorithms. 


### Overview: 

Many a times we have to deal with the text data and it is very common to find the spelling mistakes. Spelling mistakes can mean loss of information in such cases. But, thanks to simple spelling correction aglorithms like this one, we can correct the spelling mistakes with good accuracy. 

A use-case: While analysing the customer reviews for various products, we might encounter many texts with slangs or improper spellings. This algorithm can be used to treat the text before putting it through any analysis like sentiment analysis, topic modelling, etc. 


![title](Data/spelling_mistakes_meme.jpg)

### Plan of attack: 

I first plan to load the dataset that I shall be using to create a reference dictionary for correcting the spellings. Larger it is the better it is. Then, we will create the algorithm to correct the spellings.

#### Credits
The post is inspired by a post on Peter Norvig's [blog.](http://www.norvig.com/). The algorithm and some code snippets is borrowed from his blog.

Let's get started... 


In [201]:
# Import the dependencies : 
import re  # For regular expressions 
import urllib
import string

In [160]:
text= urllib.request.urlopen('https://raw.githubusercontent.com/norvig/pytudes/master/data/text/big.txt').read().decode('utf-8', 'strict')

In [161]:
print(len(text)) # such a large text!

6488666


Let's treat the text above and get rid of any punctuations numbers and special characters.Also, we will convert the text into lowercase. 

In [162]:
def tokenize_and_lowercase(text): 
    '''Convert the text into lowercase '''
    return( re.findall('[a-z]+' , text.lower()))

In [163]:
words = tokenize_and_lowercase(text)

# Here are the first few of them: 

print(words[:20])

['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'adventures', 'of', 'sherlock', 'holmes', 'by', 'sir', 'arthur', 'conan', 'doyle', 'in', 'our', 'series', 'by', 'sir']


In [164]:
# lets take the sample from this and join them:
import random
def sample_n(bag , n): 
    ''' Select n words at a random'''
    return(' '.join(random.choice(bag) for i in range(n)))


In [165]:
sample_n(words , 20)

'i countess bridge margins hard the third at general two has drop was pipes to meaning more begins bank measures'

##### Bag of words: 

One way to represent a bag of words is using dictionary style - "Counter". 

In [167]:
counts= Counter(words)
print(bag.most_common(10))

[('the', 5810), ('and', 3088), ('i', 3038), ('to', 2823), ('of', 2778), ('a', 2701), ('in', 1823), ('that', 1767), ('it', 1749), ('you', 1572)]


## Spelling Correction :

Let me take a jab at explaining the algorithm: 

**Intuition:**  
For every word, try to find out the words which are near to that word and are more likely to be the correct ones. 

That might sound little confusing. But let's try to eliminate the confusion. 

Say, we have a word - *wird*. Here, the words that are near to this can be wirdh', 'wirdw', 'jird'.....'weird', ....., etc. The one which is most likely to be found in our corpus and is nearest to it is 'weird' (which is also the right word). Our algorithm will pick it up in that case.

**How do we define near?**

We use the following 4 ways to extract the possible combinations of near words- 

| Method | Examples | 
| --- | --- | 
| deletes  | 'ird' ,  'wrd' , 'wid', 'wir'|
| transposes  | 'iwrd', 'wrid', 'widr'|
| replaces  |   'ird' ,  'wrd' , 'wid', 'wir'|
| inserts  |  'aird', 'bird', 'cird', 'dird', 'eird'|

We can also put another layer of edits, i.e. extracting the edits of the edits. 


Step 1 : Define the candidates
Step 2 :



In [187]:

# Defining all the candidates: 
def correct(word):
    "Find the best spelling correction for this word."
    # Prefer edit distance 0, then 1, then 2; otherwise default to word itself.
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=counts.get)

In [188]:
def known(words):
    "Return the subset of words that are actually in the dictionary."
    return {w for w in words if w in counts}

# The edit one
def edits0(word): 
    "Return all strings that are zero edits away from word (i.e., just word itself)."
    return {word}

# Edit 2 
def edits2(word):
    "Return all strings that are two edits away from this word."
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [209]:
# Edit 1 
def edits1(word):
    "Return all strings that are one edit away from this word."
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    "Return a list of all possible (first, rest) pairs that comprise word."
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

alphabet = string.ascii_lowercase

In [210]:
splits('spelll')

[('', 'spelll'),
 ('s', 'pelll'),
 ('sp', 'elll'),
 ('spe', 'lll'),
 ('spel', 'll'),
 ('spell', 'l'),
 ('spelll', '')]

In [211]:
print(edits0('wird'))

{'wird'}


In [212]:
print(edits1('wird'))

{'cwird', 'wire', 'wiid', 'lird', 'cird', 'kird', 'wbird', 'wirmd', 'twird', 'wuird', 'wxrd', 'wikrd', 'zwird', 'wircd', 'mird', 'wirld', 'wjrd', 'wiyd', 'wiird', 'hird', 'wirud', 'wirdv', 'wirdw', 'wyrd', 'wijd', 'werd', 'wsrd', 'ward', 'wivd', 'wirb', 'wirpd', 'wiprd', 'wirdp', 'wirvd', 'wrrd', 'wiro', 'wirs', 'widr', 'wied', 'wicd', 'fwird', 'wirw', 'wizd', 'wgird', 'wirdm', 'wrid', 'wiwrd', 'bird', 'wiad', 'wfrd', 'wprd', 'ewird', 'whird', 'wirdd', 'wirdz', 'sird', 'wirdl', 'hwird', 'ywird', 'wirsd', 'jird', 'kwird', 'wgrd', 'rird', 'wirgd', 'wxird', 'wilrd', 'vwird', 'wirdx', 'wirdo', 'wkrd', 'wdird', 'wirqd', 'wifrd', 'wirc', 'wiqrd', 'iwrd', 'wnrd', 'word', 'wibrd', 'widrd', 'wirdn', 'jwird', 'lwird', 'wlird', 'nwird', 'fird', 'wirl', 'oird', 'wiurd', 'wifd', 'wibd', 'wirrd', 'wvird', 'wirad', 'wirm', 'wirbd', 'wiyrd', 'wirzd', 'wimrd', 'wiard', 'wirxd', 'wirda', 'wirdr', 'wrird', 'witrd', 'wired', 'wqrd', 'wirdk', 'wizrd', 'waird', 'wiod', 'wigd', 'wirdf', 'wijrd', 'wirh', 'wir

In [213]:
print(len(edits2('wird')))

24254


In [214]:
def correct_text(text):
    "Correct all the words within a text, returning the corrected text."
    return re.sub('[a-zA-Z]+', correct_match, text)

def correct_match(match):
    "Spell-correct word in match, and preserve proper upper/lower/title case."
    word = match.group()
    return case_of(word)(correct(word.lower()))

def case_of(text):
    "Return the case-function appropriate for text: upper, lower, title, or just str."
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

Time to test the results...

In [216]:
correct_text('Pleaze do nat makk any mestakes')

'Please do at make any mistakes'

In [217]:
correct_text('Whaat is wrong heree? Nothingg friendd, aal is weel')

'What is wrong here? Nothing friend, all is well'

Appendix:

big.txt : It is a large text body of 6 million words. It is compiled by some texts from gutenberg and 

In [ ]:

    

pairs      = splits(word)
deletes    = [a+b[1:]           for (a, b) in pairs if b]
transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
return set(deletes + transposes + replaces + inserts)

In [184]:
from IPython.display import HTML, display

data = [[1,2,3],
        [4,5,6],
        [7,8,9],
        ]

display(HTML(
   '<table><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
       )
))

1,2,3
4,5,6
7,8,9


| Stretch/Untouched | ProbDistribution | Accuracy |
| --- | --- | --- |
| Stretched | Gaussian | .843 |

In [203]:
string.ascii_lowercase

'abcdefghijklmnopqrstuvwxyz'